# Example: Compute the Primal and Dual Solution for a Reaction Flow Problem
This example will familiarize students with using [linear programming](https://en.wikipedia.org/wiki/Linear_programming) to compute the `reaction flow` or `flux` through a chemical reaction network. 

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-12/L12a/Manifest.toml`


## Prerequisites 
Before we can compute the primal and dual solutions to the flux problem, we need to load the stoichiometric matrix from the reaction file, in this case [Toy.net](data/Toy.net), i.e., the same example reaction network we used for `PS3`.

* Our first step is to load the list of reactions. This is made easy with the `readreactionfile(...)` function. It takes the path to the reaction file, along with other arguments like comment characters and the delimiter character. The function then returns the reactions in a dictionary `R.`
* From the reactions, we'll create the stoichiometric matrix model, stored in the variable `S,` specify which unknowns (columns) are measured and which are estimated. Then we'll construct the system matrix `Â` and the measurement matrix `B.`

In [2]:
# Load the reaction file -
path_to_reaction_file = joinpath(_PATH_TO_DATA, "Toy.net");
R = readreactionfile(path_to_reaction_file, comment="//", delim=',', expand = false);

# build the stoichiometric matrix -
S = build(MyStoichiometricMatrixModel, R);
number_of_species = length(S.species)

measured_columns = [3]; # we have measured these columns
unkown_columns = setdiff(1:24, measured_columns); # we want to estimate these columns

# build the system matrix -
d = [1,1,-1]
A = [S.matrix d[1]*diagm(ones(number_of_species)) d[2]*diagm(ones(number_of_species)) d[3]*diagm(ones(number_of_species))];

Â = A[:,unkown_columns];
B = A[:,measured_columns];

Next, we'll need to specify values for the measurements. These values must be in the same order as the `measured_columns` array. Store this data in the `measurement` array:

In [3]:
measurement = [1.0]; # we want a unit of product formation

Finally, we'll create a list of stream labels in the `stream_label_vector` so we can make a table of the estimated solution later:

In [4]:
stream_label_vector = [
    "ϵ̇₁", "ϵ̇₂", "ϵ̇₃", 
    "ṅ_A1_1", "ṅ_A2_1", "ṅ_B_1", "ṅ_C_1", "ṅ_P_1", "ṅ_x_1", "ṅ_y_1",
    "ṅ_A1_2", "ṅ_A2_2", "ṅ_B_2", "ṅ_C_2", "ṅ_P_2", "ṅ_x_2", "ṅ_y_2",
    "ṅ_A1_3", "ṅ_A2_3", "ṅ_B_3", "ṅ_C_3", "ṅ_P_3", "ṅ_x_3", "ṅ_y_3"
];
label_vector = stream_label_vector[unkown_columns];

In [5]:
(number_of_species, number_of_flows) = size(Â);

## `Primal`: Compute the primal flux solution
Let's solve the `primal` linear programming problem for the unknown values in our problem, i.e., the unmeasured mole flow rates and the open extents of reaction (assume there are `u` unmeasured values). The problem we are solving is a linear programming problem of the form:

$$
\begin{eqnarray*}
\text{maximize}~\mathcal{O}(\mathbf{x}) &=& \sum_{i=1}^{u} c_{i}\cdot{x}_{i}\\
\text{subject to}~\hat{\mathbf{A}}\cdot\mathbf{x} &\leq&-\mathbf{B}\cdot\dot{\mathbf{m}}\\
\text{and}~0\leq{x_{i}}&\leq&{U_{i}}\qquad{i=1,2,\dots,u}
\end{eqnarray*}
$$

where $x_{i}$ represent the unknown flow rates and open extents, $\hat{\mathbf{A}}$ is system matrix holding the unknown columns, $\mathbf{B}$ is the measurement matrix (measured columns), $\dot{\mathbf{m}}$ is the measurement vector and $U_{i}$ is an _upper bound_ on each of the unknown flows. 
* The values of the coefficients in the objective function $c_{i}$ are specified by you to represent different problems. For example, suppose we wanted to __minimize__ the material that was required to meet a specified production target.

In [6]:
c = ones(number_of_flows); # we want to *minimize* the material usage. What?!?

### Setup the bounds on the unknown flows
When dealing with unknown flow values in a system, specifying the lower and upper bounds for each is essential. These bounds are typically determined using physical or chemical reasoning. For example, we know that mole flows must be non-negative, or there is some maximum limit to a reaction rate, etc. 
* These values can be stored in an array named `bounds_primal.` The first column of the array corresponds to the lower bound, while the second column corresponds to the upper bound for the unknown variable $x_{i}$.

In [7]:
bounds_primal = [

    # --- reactions ---
    0.0 100.0 ; # 1 1 ϵ̇₁
    0.0 100.0 ; # 2 ϵ̇₂ 
    # 0.0 100.0 ; # 3 1 ϵ̇₃ we measured this
    
    # --- stream 1 -----
    0.0 10.0 ; # 4 s1 A1
    0.0 6.0  ; # 5 s1 A2
    0.0 0.0  ; # 6 s1 B
    0.0 0.0  ; # 7 s1 C
    0.0 0.0  ; # 8 s1 P
    0.0 0.0  ; # 9 s1 x
    0.0 0.0  ; # 10 s1 y

    # --- stream 2 -----
    0.0 0.0  ; # 11 s2 A1
    0.0 0.0  ; # 12 s2 A2
    0.0 0.0  ; # 13 s2 B
    0.0 0.0  ; # 14 s2 C
    0.0 0.0  ; # 15 s2 P
    0.0 10.0  ; # 16 s2 x
    0.0 10.0  ; # 17 s2 y

    # --- stream 3 (unbounded)
    0.0 1000.0  ; # 18 s2 A1
    0.0 1000.0  ; # 19 s2 A2
    0.0 1000.0  ; # 20 s2 B
    0.0 1000.0  ; # 21 s2 C
    0.0 1000.0  ; # 22 s2 P
    0.0 1000.0  ; # 23 s2 x
    0.0 1000.0  ; # 24 s2 y
];

Next, we create an instance of the `MyLinearProgrammingProblemModel` model using a `build(...)` method and store this model in the `primal_problem` variable. 
* This model holds the data associated with the problem, e.g., the unknown system matrix `Â,` the right-hand side vector `-B*measurement,` the problem bounds, and the objective coefficients `c.`

In [8]:
primal_problem = build(MyLinearProgrammingProblemModel, (
    c = c,
    A = Â,
    b = -B*measurement,
    lb = bounds_primal[:,1],
    ub = bounds_primal[:,2]
));

Finally, we pass the `primal_problem` variable to the `solve(...)` method, which constructs the linear program using the [JuMP domain-specific language](https://jump.dev/). 
* The implementation of the `solve(...)` method is in the [src/Solver.jl](src/Solver.jl) file. It takes the data from the `primal_problem` instance, builds the various problem structures and returns the solution in a dictionary.

In [9]:
primal_soln = solve(primal_problem)

Dict{String, Any} with 2 entries:
  "argmax"          => [10.0, 10.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  … …
  "objective_value" => 92.0

`Unhide` the code block below to see how we make a table holding the `non-zero` flow values that we estimated using linear programming.

In [10]:
primal_soln_table = DataFrame();
primal_flow = primal_soln["argmax"]; # get the primal solution from the solution dictionary
for i ∈ eachindex(unkown_columns)

    if (primal_flow[i] != 0.0)
        row_data = (
            label=label_vector[i],
            value = primal_flow[i]
        );
        push!(primal_soln_table, row_data);
    end
end
primal_soln_table

Row,label,value
,String,Float64
1,ϵ̇₁,10.0
2,ϵ̇₂,10.0
3,ṅ_A1_1,10.0
4,ṅ_A2_1,6.0
5,ṅ_x_2,10.0
6,ṅ_y_2,10.0
7,ṅ_A2_3,5.0
8,ṅ_C_3,1.0
9,ṅ_P_3,10.0
